In [6]:
## Install libraries
! python -m pip install --upgrade pip --quiet
! pip install spacy --quiet
! pip3 install spacy-transformers --quiet
! python -m spacy download en_core_web_trf --quiet
# ! python -m spacy download en_trf_bertbaseuncased_lg --quiet

2021-06-29 04:44:16.249316: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-06-29 04:44:16.249394: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-06-29 04:44:16.249407: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
2021-06-29 04:44:48.675869: W tensorflow/stream_exe

In [25]:
# Import spacy
import spacy

nlp_trf = spacy.load("en_core_web_trf")

# Feed an example sentence to the model; store output under 'example_doc'
example_doc = nlp_trf("Helsinki is the capital of Finland.")

# Check the length of the Doc object
example_doc.__len__()

7

In [26]:
# Check the type of the 'trf_data' object using the type() function
type(example_doc._.trf_data)

spacy_transformers.data_classes.TransformerData

In [27]:
# Check the shape of the first item in the list
example_doc._.trf_data.tensors[0].shape

(1, 11, 768)

In [28]:
# Check the shape of the first item in the list
example_doc._.trf_data.tensors[1].shape

(1, 768)

In [29]:
# Check the first ten dimensions of the tensor
example_doc._.trf_data.tensors[0][0][:10]

array([[ 0.08683846, -0.7730738 , -1.1122073 , ...,  0.44053328,
         0.36458606, -0.48721927],
       [-1.3220197 , -0.58153176,  0.45662835, ..., -0.4110549 ,
        -1.5180305 , -0.871696  ],
       [-1.110841  , -0.58721155,  0.2594491 , ..., -0.46206427,
        -1.4726617 , -0.8277549 ],
       ...,
       [-0.30590406, -0.46042696, -0.08148608, ...,  2.4376936 ,
         0.68996066, -1.0034345 ],
       [-0.02552489, -0.950257  , -0.62682885, ...,  0.5177102 ,
         0.18616802,  0.41015333],
       [ 0.17655824, -0.5720697 , -1.3821042 , ...,  0.25179133,
         0.3483243 , -0.09476944]], dtype=float32)

In [30]:
# Access the Transformer tokens under the key 'input_texts'
example_doc._.trf_data.tokens['input_texts']

[['<s>',
  'H',
  'els',
  'inki',
  'Ġis',
  'Ġthe',
  'Ġcapital',
  'Ġof',
  'ĠFinland',
  '.',
  '</s>']]

In [31]:
example_doc[0], example_doc._.trf_data.align[0].data

(Helsinki, array([[1],
        [2],
        [3]], dtype=int32))

In [32]:
# Import the Language object under the 'language' module in spaCy,
# and NumPy for calculating cosine similarity.
from spacy.language import Language
import numpy as np

# We use the @ character to register the following Class definition
# with spaCy under the name 'tensor2attr'.
@Language.factory('tensor2attr')

# We begin by declaring the class name: Tensor2Attr. The name is 
# declared using 'class', followed by the name and a colon.
class Tensor2Attr:
    
    # We continue by defining the first method of the class, 
    # __init__(), which is called when this class is used for 
    # creating a Python object. Custom components in spaCy 
    # require passing two variables to the __init__() method:
    # 'name' and 'nlp'. The variable 'self' refers to any
    # object created using this class!
    def __init__(self, name, nlp):
        
        # We do not really do anything with this class, so we
        # simply move on using 'pass' when the object is created.
        pass

    # The __call__() method is called whenever some other object
    # is passed to an object representing this class. Since we know
    # that the class is a part of the spaCy pipeline, we already know
    # that it will receive Doc objects from the preceding layers.
    # We use the variable 'doc' to refer to any object received.
    def __call__(self, doc):
        
        # When an object is received, the class will instantly pass
        # the object forward to the 'add_attributes' method. The
        # reference to self informs Python that the method belongs
        # to this class.
        self.add_attributes(doc)
        
        # After the 'add_attributes' method finishes, the __call__
        # method returns the object.
        return doc
    
    # Next, we define the 'add_attributes' method that will modify
    # the incoming Doc object by calling a series of methods.
    def add_attributes(self, doc):
        
        # spaCy Doc objects have an attribute named 'user_hooks',
        # which allows customising the default attributes of a 
        # Doc object, such as 'vector'. We use the 'user_hooks'
        # attribute to replace the attribute 'vector' with the 
        # Transformer output, which is retrieved using the 
        # 'doc_tensor' method defined below.
        doc.user_hooks['vector'] = self.doc_tensor
        
        # We then perform the same for both Spans and Tokens that
        # are contained within the Doc object.
        doc.user_span_hooks['vector'] = self.span_tensor
        doc.user_token_hooks['vector'] = self.token_tensor
        
        # We also replace the 'similarity' method, because the 
        # default 'similarity' method looks at the default 'vector'
        # attribute, which is empty! We must first replace the
        # vectors using the 'user_hooks' attribute.
        doc.user_hooks['similarity'] = self.get_similarity
        doc.user_span_hooks['similarity'] = self.get_similarity
        doc.user_token_hooks['similarity'] = self.get_similarity
    
    # Define a method that takes a Doc object as input and returns 
    # Transformer output for the entire Doc.
    def doc_tensor(self, doc):
        
        # Return Transformer output for the entire Doc. As noted
        # above, this is the last item under the attribute 'tensor'.
        # Average the output along axis 0 to handle batched outputs.
        return doc._.trf_data.tensors[-1].mean(axis=0)
    
    # Define a method that takes a Span as input and returns the Transformer 
    # output.
    def span_tensor(self, span):
        
        # Get alignment information for Span. This is achieved by using
        # the 'doc' attribute of Span that refers to the Doc that contains
        # this Span. We then use the 'start' and 'end' attributes of a Span
        # to retrieve the alignment information. Finally, we flatten the
        # resulting array to use it for indexing.
        tensor_ix = span.doc._.trf_data.align[span.start: span.end].data.flatten()
        
        # Fetch Transformer output shape from the final dimension of the output.
        # We do this here to maintain compatibility with different Transformers,
        # which may output tensors of different shape.
        out_dim = span.doc._.trf_data.tensors[0].shape[-1]
        
        # Get Token tensors under tensors[0]. Reshape batched outputs so that
        # each "row" in the matrix corresponds to a single token. This is needed
        # for matching alignment information under 'tensor_ix' to the Transformer
        # output.
        tensor = span.doc._.trf_data.tensors[0].reshape(-1, out_dim)[tensor_ix]
        
        # Average vectors along axis 0 ("columns"). This yields a 768-dimensional
        # vector for each spaCy Span.
        return tensor.mean(axis=0)
    
    # Define a function that takes a Token as input and returns the Transformer
    # output.
    def token_tensor(self, token):
        
        # Get alignment information for Token; flatten array for indexing.
        # Again, we use the 'doc' attribute of a Token to get the parent Doc,
        # which contains the Transformer output.
        tensor_ix = token.doc._.trf_data.align[token.i].data.flatten()
        
        # Fetch Transformer output shape from the final dimension of the output.
        # We do this here to maintain compatibility with different Transformers,
        # which may output tensors of different shape.
        out_dim = token.doc._.trf_data.tensors[0].shape[-1]
        
        # Get Token tensors under tensors[0]. Reshape batched outputs so that
        # each "row" in the matrix corresponds to a single token. This is needed
        # for matching alignment information under 'tensor_ix' to the Transformer
        # output.
        tensor = token.doc._.trf_data.tensors[0].reshape(-1, out_dim)[tensor_ix]

        # Average vectors along axis 0 (columns). This yields a 768-dimensional
        # vector for each spaCy Token.
        return tensor.mean(axis=0)
    
    # Define a function for calculating cosine similarity between vectors
    def get_similarity(self, doc1, doc2):
        
        # Calculate and return cosine similarity
        return np.dot(doc1.vector, doc2.vector) / (doc1.vector_norm * doc2.vector_norm)

In [33]:
# Add the component named 'tensor2attr', which we registered using the
# @Language decorator and its 'factory' method to the pipeline.
nlp_trf.add_pipe('tensor2attr')

# Call the 'pipeline' attribute to examine the pipeline
nlp_trf.pipeline

[('transformer',
  <spacy_transformers.pipeline_component.Transformer at 0x7f59805d6048>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x7f598a4e5780>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7f5980436590>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f59804366c8>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7f59802b4b08>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x7f59802b4fc8>),
 ('tensor2attr', <__main__.Tensor2Attr at 0x7f5994478128>)]

In [112]:
# Define two example sentences and process them using the Transformer-based
# language model under 'nlp_trf'.
doc_city_trf = nlp_trf("the capital town of usa is washinton")
doc_money_trf = nlp_trf("our capital city is taipei.")

In [106]:
# Retrieve vectors for the two Tokens corresponding to "capital";
# assign to variables 'city_trf' and 'money_trf'.
city_trf = doc_city_trf[5]
money_trf = doc_money_trf[0]

# Compare the similarity of the two meanings of 'capital'
city_trf.similarity(money_trf)

0.12347994

In [120]:
doc_city_trf.similarity(doc_money_trf)


0.9453276614922332

In [129]:
doc_city_trf._.trf_data

TransformerData(wordpieces=WordpieceBatch(strings=[['<s>', 'the', 'Ġcapital', 'Ġtown', 'Ġof', 'Ġus', 'a', 'Ġis', 'Ġwash', 'inton', '</s>']], input_ids=array([[    0,   627,   812,  1139,     9,   201,   102,    16, 10397,
        10528,     2]]), attention_mask=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), lengths=[11], token_type_ids=None), tensors=[array([[[ 0.24946997, -0.37220186, -0.7267498 , ...,  0.06434161,
          0.16448084, -0.4385782 ],
        [-0.1141268 , -1.1301125 ,  0.05209627, ..., -1.0581224 ,
         -0.38165125, -1.1613958 ],
        [-0.7097907 , -0.89201367,  0.56712574, ...,  0.6758817 ,
          0.4737274 ,  0.14610694],
        ...,
        [ 0.18009594, -0.14503776, -0.05266785, ..., -1.2241157 ,
         -0.21417822,  0.74991214],
        [ 0.2771276 , -0.15657264, -0.2266963 , ..., -1.540317  ,
         -0.16685581,  0.63955176],
        [ 0.3855707 , -0.4916963 , -1.1064452 , ...,  0.13321239,
          0.26810056, -0.46347734]]], dtype=float32), array(

In [131]:
len(doc_city_trf._.trf_data.tensors[-1].mean(axis=0))

768

In [36]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering_question(data,NUM_CLUSTERS = 15):

    sentences = data['text']

    X = np.array(data['emb'].tolist())

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    data['cluster'] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters

In [37]:
def distance_from_centroid(row):
    # type of emb and centroid is different, hence using tolist below
    return distance_matrix([row['emb']], [row['centroid'].tolist()])[0][0]

# Compute centroid distance to the data
data['distance_from_centroid'] = data.apply(distance_from_centroid, axis=1)

NameError: name 'data' is not defined

In [118]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans# 隨機產生 10 組 2 features 的資料 500 筆 (dy 即原始目標值 label 0~9)
dx, dy = make_blobs(n_samples=500, n_features=2, centers=10, random_state=42)# 用 KMeans 在資料中找出 5 個分組
kmeans = KMeans(n_clusters=5)
kmeans.fit(dx)# 預測新的目標值 label
new_dy = kmeans.predict(dx)
